# Hypothetical Hybrid GNN-QUBO Alignment Pipeline

Keep in mind that this Jupyter notebook's purpose is not to show an advantage of the QUBO method over the NN method. 

This is just a proposed pipeline for QUBO re-ranking. 

It implements all of the steps, from data fetching to the QUBO solver.

We start by building two regular knowledge graphs (unpruned) and then prune them to simulate how a smaller knowledge graph would look like (in theory, that smaller knowledge graph would contain the ambiguous entities which didn't get a high alignment confidence score).

## Section 1: Load Project Dependencies
- Set up the Python path for the project and import every module that the pipeline relies on.

In [1]:
from pathlib import Path
import sys
from types import SimpleNamespace
import webbrowser

import pandas as pd
from IPython.display import HTML, IFrame, display

repo_root = Path().resolve().parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from src.config import *
from src.evaluation.solvers import (
    solve_alignment_with_annealer,
    solve_alignment_with_nearest_neighbor,
 )
from src.kg_construction.fetch_data import fetch_wiki_data, fetch_arxiv_data
from src.kg_construction.build_kg import build_unpruned_kgs, prune_kgs
from src.embedding.generate_embeddings import (
    generate_relation_embeddings,
    generate_entity_embeddings,
 )
from src.utils.graph_visualizer import visualize_ttl

# [FIX THIS LATER] maintain backward-compatible variable name for existing cells
ALIGNED_ENTITIES_CSV = ALIGNED_ENTITIES_ANNEALER_CSV

pd.set_option("display.max_colwidth", None)

# create directories if they don't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
KG_DIR.mkdir(parents=True, exist_ok=True)
EMBEDDINGS_DIR.mkdir(parents=True, exist_ok=True)
ENTITIES_DIR.mkdir(parents=True, exist_ok=True)
WIKI_ENTITIES_DIR.mkdir(parents=True, exist_ok=True)
ARXIV_ENTITIES_DIR.mkdir(parents=True, exist_ok=True)

def _open_in_browser(path, title):
    """Open a local HTML file in the default browser."""
    path = Path(path)
    webbrowser.open(f"file://{path.resolve()}")
    display(HTML(f"<p><i>Opening '{title}' in the browser...</i></p>"))

/home/nuno/python_envs/KGA/lib/python3.11/site-packages/torch/cuda/__init__.py:827: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/nuno/python_envs/KGA/lib/python3.11/site-packages/torch/cuda/__init__.py:1034: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:119.)
  r = torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count
/home/nuno/python_envs/KGA/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nuno/python_envs/KGA/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. P

## Section 2: Data Preparation
- Fetch the Wikipedia and arXiv raw data.  

In [2]:
wiki_titles = []
arxiv_ids = []


print("Fetching source corpora...")
wiki_summaries, wiki_titles = fetch_wiki_data()
arxiv_abstracts, arxiv_ids = fetch_arxiv_data()
print(f"Wikipedia titles: {wiki_titles}")
print(f"arXiv IDs: {arxiv_ids}")

Fetching source corpora...
-> all requested Wikipedia articles already cached; reusing local files
    -> returning 10 Wikipedia summaries.
-> all requested arXiv abstracts already cached; reusing local files
    -> returning 10 arXiv abstracts.
Wikipedia titles: ['Quantum algorithm', 'Post-quantum cryptography', 'Quantum optimization algorithms', 'Quantum computing', "Shor's algorithm", "Grover's algorithm", 'Noisy intermediate-scale quantum computing', 'Quantum machine learning', 'Quantum counting algorithm', 'Quantum phase estimation algorithm']
arXiv IDs: ['2310.03011v2', '2406.13258v3', '2312.13636v3', '0708.0261v1', 'quant-ph/9508027v2', '2108.10854v2', '1801.00862v3', '1611.09347v2', 'quant-ph/9805082v1', 'quant-ph/9511026v1']


# Section 3: Build the KGs.

- Run the NLP pipeline to perform Named Entity Recognition (NER) and Relationship Extraction (RE) on the raw data.

- Use the entities and the relations between them to build two large unpruned graphs.

- Take the unpruned graphs and reduce them to just a couple entities. In practice, those would be tha "ambiguous" entities whose alignment confidence score is low.

In [3]:
print("\nBuilding and pruning knowledge graphs...")
build_unpruned_kgs(
    wiki_data=wiki_summaries, 
    arxiv_data=arxiv_abstracts
)

visualize_ttl(KG_WIKI_UNPRUNED_PATH, KG_DIR / "unpruned_wiki_kg.html")
visualize_ttl(KG_ARXIV_UNPRUNED_PATH, KG_DIR / "unpruned_arxiv_kg.html")


_open_in_browser(KG_DIR / "unpruned_wiki_kg.html", "Unpruned Wiki Knowledge Graph")
_open_in_browser(KG_DIR / "unpruned_arxiv_kg.html", "Unpruned arXiv Knowledge Graph")



Building and pruning knowledge graphs...

--- STEP 1: SKIPPING BUILD; UNPRUNED TTLs ARE ALREADY PRESENT ---
loading graph from: /home/nuno/Documents/QUBO-KGA/output/KGs/kg_wiki_unpruned.ttl

saved graph visualization to: /home/nuno/Documents/QUBO-KGA/output/KGs/unpruned_wiki_kg.html
loading graph from: /home/nuno/Documents/QUBO-KGA/output/KGs/kg_arxiv_unpruned.ttl

saved graph visualization to: /home/nuno/Documents/QUBO-KGA/output/KGs/unpruned_arxiv_kg.html


In [4]:
prune_kgs()

wiki_html = KG_DIR / "pruned_wiki_kg.html"
arxiv_html = KG_DIR / "pruned_arxiv_kg.html"
visualize_ttl(KG_WIKI_FINAL_PATH, wiki_html)
visualize_ttl(KG_ARXIV_FINAL_PATH, arxiv_html)

# Open HTML files in browser
_open_in_browser(wiki_html, "Pruned Wiki Knowledge Graph")
_open_in_browser(arxiv_html, "Pruned arXiv Knowledge Graph")


--- STEP 4: PRUNING KGs ---

-> pruning Wiki KG...
    -> loaded 145 raw triples, pruning with 9 entities.
    -> Found and added 9 matching entities.
    -> Saving 14 clean triples to /home/nuno/Documents/QUBO-KGA/output/KGs/kg_wiki_final.ttl

-> pruning arXiv KG...
    -> loaded 108 raw triples, pruning with 9 entities.
    -> Found and added 9 matching entities.
    -> Saving 10 clean triples to /home/nuno/Documents/QUBO-KGA/output/KGs/kg_arxiv_final.ttl
loading graph from: /home/nuno/Documents/QUBO-KGA/output/KGs/kg_wiki_final.ttl

saved graph visualization to: /home/nuno/Documents/QUBO-KGA/output/KGs/pruned_wiki_kg.html
loading graph from: /home/nuno/Documents/QUBO-KGA/output/KGs/kg_arxiv_final.ttl

saved graph visualization to: /home/nuno/Documents/QUBO-KGA/output/KGs/pruned_arxiv_kg.html


## Section 4: Generate Embeddings
Generate the following embeddings:
- Entity embeddings (using a GAE that fine-tunes the SciBERT embeddings)
- Relation embeddings (using the SciBERT embeddings)

In [5]:
RUN_EMBEDDINGS = True

if RUN_EMBEDDINGS:
    print("Generating relation embeddings...")
    generate_relation_embeddings()
    print("\nGenerating entity embeddings...")
    generate_entity_embeddings()
else:
    print("Skipping embedding generation; using cached tensors.")



Generating relation embeddings...

--- Part 1: Generating Relation Embeddings (for H_structure) ---
Loading SciBERT model: allenai/scibert_scivocab_cased...
Discovered 175 relation labels.
  - aim for
  - aims at developing
  - also known as
  - analyze
  - analyzes
  - applied to
  - applies to
  - approximately executes
Discovered 175 relation labels.
  - aim for
  - aims at developing
  - also known as
  - analyze
  - analyzes
  - applied to
  - applies to
  - approximately executes
  - based on
  - be impacted by
  - become more relevant
  - believed to be
  - can be performed on
  - can bruteforce
  - can exist in
  - can simulate
  - based on
  - be impacted by
  - become more relevant
  - believed to be
  - can be performed on
  - can bruteforce
  - can exist in
  - can simulate
  - can solve faster than
  - cannot be efficiently simulated on
  - challenges
  - characterized by
  - class of
  - coined by
  - coined in year
  - combine
  - can solve faster than
  - cannot be effi

## Section 5: Formulate the problem as a QUBO and solve it
Perform the QUBO formulation:
$$
H_{total} = \underbrace{\sum_{i,a}{-S(i, a) \cdot x_{i,a}}}_{H_{\text{node}}} + \underbrace{\sum_{i,j,a,b}{-w_{ij,ab} \cdot x_{i,a} \cdot x_{j,b}}}_{H_{\text{structure}}} + \underbrace{\sum_{i} P_{1} \sum_{a=1}^M \sum_{b=a+1}^M x_{i,a} x_{i,b}}_{\text{Constraint}\ 1} + \underbrace{\sum_{a} P_{2} \sum_{i=1}^N \sum_{j=i+1}^N x_{i,a} x_{j,a}}_{\text{Constraint}\ 2}
$$

- Where:
  - $x_{i,a}$: A binary variable (1 or 0) that is 1 if we align entity $i$ from KG1 with entity $a$ from KG2.
  - $S(i,a)$: The similarity score between entity $i$ and $a$, derived from the GAE embeddings.
  - $w_{ij,ab}$: The structural similarity weight, derived from the SciBERT relation embeddings.
  - $P_1, P_2$: Large positive penalty constants to enforce the constraints.
  - Constraint 1: Enforces that each entity $i$ in KG1 maps to at most one entity in KG2. If $i$ matches zero entities, the penalty is 0. If it matches one, the penalty is 0. If it matches two or more, the penalty is high.
  - Constraint 2: Enforces that each entity $a$ in KG2 is mapped to by at most one entity from KG1. This allows entities to remain unaligned, making the formulation more robust to realistic KGs that do not have perfect 1-to-1 overlap.

And solve it using quantum annealing (more details in the README.md file).

In [6]:
SOLVE_QUBO = True

result = None
if SOLVE_QUBO:
    print("\nSolving the alignment QUBO...")
    result = solve_alignment_with_annealer(
        similarity_threshold=DEFAULT_SIMILARITY_THRESHOLD,
        max_structural_pairs=2000,
        visualize=True
    )
else:
    print("Skipping QUBO solve; falling back to existing artefacts.")

if result is None:
    result = SimpleNamespace(
        alignments=[],
        energy=float("nan"),
        sampleset=None,
        aligned_graph_path=KG_ALIGNED_PATH,
        aligned_graph_html=(KG_DIR / "kg_aligned.html"),
        alignment_report_path=ALIGNED_ENTITIES_ANNEALER_CSV,
    )


Solving the alignment QUBO...
[QUBO] candidate variables: 27, structural pairs: 5
[QUBO] matrix exported to /home/nuno/Documents/QUBO-KGA/output/qubo/qubo_matrix.csv with dimension 27×27
[QUBO] matrix exported to /home/nuno/Documents/QUBO-KGA/output/qubo/qubo_matrix_H_node.csv with dimension 27×27
[QUBO] matrix exported to /home/nuno/Documents/QUBO-KGA/output/qubo/qubo_matrix_H_structure.csv with dimension 27×27
[QUBO] matrix exported to /home/nuno/Documents/QUBO-KGA/output/qubo/qubo_matrix_H_penalty.csv with dimension 27×27
[QUBO] running simulated annealer with num_reads=100, beta_range=None, seed=None
[QUBO] best sample energy=-6.4590 produced 7 alignments
loading graph from: /home/nuno/Documents/QUBO-KGA/output/KGs/kg_aligned.ttl

saved graph visualization to: /home/nuno/Documents/QUBO-KGA/output/KGs/aligned_kg.html
[QUBO] alignment report saved to /home/nuno/Documents/QUBO-KGA/output/alignments/alignment_annealer.csv with 7 matches and 4 unaligned entries


## Section 6: Display HTML Visualizations
Render the pruned graphs, aligned knowledge graph, and alignment report directly within the notebook.

In [7]:
aligned_html = KG_DIR / "kg_aligned.html"
# create the visualizations

visualize_ttl(KG_ALIGNED_PATH, aligned_html)

_open_in_browser(aligned_html, "Aligned Knowledge Graph")

# # Display only the DataFrame in the notebook
display(pd.read_csv(ALIGNED_ENTITIES_ANNEALER_CSV))

loading graph from: /home/nuno/Documents/QUBO-KGA/output/KGs/kg_aligned.ttl

saved graph visualization to: /home/nuno/Documents/QUBO-KGA/output/KGs/kg_aligned.html


,wiki_entity,arxiv_entity,not_aligned
0,Post-quantum cryptography,post-quantum cryptography,NaN
1,quantum machine learning,quantum machine learning,NaN
2,qubit,qubits,NaN
3,Shor's algorithm,Shor's quantum algorithms,NaN
4,Quantum optimization algorithms,quantum computing,NaN
5,Grover's algorithm,Deutsch's algorithm,NaN
6,quantum computer technology,Grover algorithm,NaN
7,NaN,NaN,wiki: Noisy intermediate-scale quantum NISQ computing
8,NaN,NaN,wiki: Peter Shor
9,NaN,NaN,arxiv: NISQ devices
